## New Lick sensor with enhanced detection in expense of noise production (optimized for behavior in 426 rigs)

- Check the pattern of licks and the time interval distribution
- Check the sniff sensor data to detect shorting

5/14 - Installation test with 690164

5/15 - Installed in box 5B, animals that were trained with it: 716457, 707349, 713545

5/16 - Swapped from 5B to 4D, animals that were trained with it: 716456, 716458, 716455

In [ ]:
import sys
sys.path.append('../../src/')

import os
from typing import Dict
from os import PathLike
from pathlib import Path

import data_io

from harp.reader import create_reader

from utils import breathing_signal as lib
from utils import analysis_utils as analysis
from utils import processing
from utils import plotting_utils as plotting

# Plotting libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
from tkinter import font
from matplotlib.gridspec import GridSpec

from matplotlib.ticker import FuncFormatter, MaxNLocator, FixedLocator
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
from scipy.signal import butter, filtfilt

def format_func(value, tick_number):
    return f"{value:.0f}"

from typing import Literal, Tuple

sns.set_context('talk')

import ipywidgets as widgets
from IPython.display import display
from matplotlib.patches import Rectangle

import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

pdf_path = r'Z:\scratch\vr-foraging\sessions'
base_path = 'Z:/scratch/vr-foraging/data/'
foraging_figures = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents\Meeting presentations\Foraging meeting\20240506\figures'

In [ ]:
pdf_path = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents\VR foraging\Data\figures'

date = datetime.date.today()
date_string = "05/15/2024"
date = datetime.datetime.strptime(date_string, "%m/%d/%Y").date()

mouse = '716456'


In [ ]:
session_found = False

directory = os.path.join(base_path, mouse)
files = os.listdir(os.path.join(base_path, mouse))

sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(directory, x)), reverse=True)

for file_name in sorted_files:

    if session_found == True:
        break
    
    print(file_name)
    session_path = os.path.join(base_path, mouse, file_name)
    session = file_name[:8]
    session_path = Path(session_path)
    
    if datetime.date.fromtimestamp(os.path.getctime(session_path)) != date:
        continue
    else:
        print('correct date found')
        session_found = True

data = analysis.load_session_data(session_path)
try:
    data['harp_olfactometer'].streams.OdorValveState.load_from_file()
    data['harp_olfactometer'].streams.EndValveState.load_from_file()
except:
    pass

try:
    data['harp_behavior'].streams.OutputSet.load_from_file()
    data['harp_behavior'].streams.OutputClear.load_from_file()
    data['config'].streams['TaskLogic'].load_from_file()
except:
    pass
reward_sites, active_site, encoder_data, config =  analysis.parse_data(data)

# patch_limit = analysis.choose_cut(reward_sites, 10)
# reward_sites = reward_sites.loc[reward_sites.active_patch <= patch_limit]

data['software_events'].streams.PatchRewardProbability.load_from_file()
preward = data['software_events'].streams.PatchRewardProbability.data['data'].values

for index, row in reward_sites.iterrows():
    reward_sites.loc[index, 'reward_probability'] = preward[0]
    if row['reward_delivered'] == 0 and row['has_choice'] == True:
        pass
    elif row['reward_delivered'] == 1 and row['has_choice'] == True:
        preward = preward[1:]
    else:
        preward = preward[1:]
        
index = reward_sites.index[1:].tolist()
index.append(0)
reward_sites['next_odor'] = index

index = reward_sites['odor_offset'].iloc[:-1].tolist()
index.insert(0, 0)
reward_sites['previous_odor'] = index


In [ ]:
## Breathing
data['harp_sniffsensor'].streams.RawVoltage.load_from_file()
breathing = pd.DataFrame(index = data['harp_sniffsensor'].streams.RawVoltage.data['RawVoltage'].index, columns=['data'])
breathing['data'] = data['harp_sniffsensor'].streams.RawVoltage.data['RawVoltage'].values
filtered=False

In [ ]:
filtered = True
t = breathing.index
signal = breathing.data

filtered_breathing = breathing.copy()

# Define a high-pass filter
def highpass_filter(data, cutoff_freq, fs, order=5):
    nyquist_freq = 0.5 * fs
    normal_cutoff = cutoff_freq / nyquist_freq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

def lowpass_filter(data, cutoff_freq, fs, order=5):
    nyquist_freq = 0.5 * fs
    normal_cutoff = cutoff_freq / nyquist_freq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

# Define filter parameters
cutoff_freq = 2  # cutoff frequency in Hz
fs = 250  # sampling frequency in Hz

# Apply high-pass filter
filtered_signal = highpass_filter(signal, cutoff_freq, fs)

cutoff_freq = 40  # cutoff frequency in Hz
filtered_signal = lowpass_filter(filtered_signal, cutoff_freq, fs)

filtered_breathing['data'] = filtered_signal


In [ ]:
data['harp_behavior'].streams.OutputSet.load_from_file()
data['harp_behavior'].streams.PulseSupplyPort0.load_from_file() # Duration of each pulse
data['harp_behavior'].streams.DigitalInputState.load_from_file()

# Find reward sites
sites = data['software_events'].streams.ActiveSite.data

zero_index = sites.index[0]
last_site = sites.index[-1] - zero_index

# Find ChoiceFeedback events (i.e. successful stops)
choice_feedback = data['software_events'].streams.ChoiceFeedback.data

# Check for licks
## mask for digital inputs

if 'harp_lickometer' in data:
    data['harp_lickometer'].streams.LickState.load_from_file()
    lick_onset = data['harp_lickometer'].streams.LickState.data['Channel0'] == True
    lick_onset = lick_onset.loc[lick_onset == True]
else:
    di_state = data['harp_behavior'].streams.DigitalInputState.data['DIPort0']
    lick_onset = di_state.loc[di_state == True]

# Find give reward event
give_reward = data['harp_behavior'].streams.OutputSet.data[['SupplyPort0']]
give_reward = give_reward.loc[give_reward.SupplyPort0 == True]

# Find hardware reward events
pulse_duration = data['harp_behavior'].streams.PulseSupplyPort0.data
valve_output_pulse = data['harp_behavior'].streams.OutputSet.data['DOPort0']

# # odor inputs
# odor_on = reward_sites.odor_onset.values
# odor_off = reward_sites.odor_offset.values

# Successfull waits
data['software_events'].streams.WaitRewardOutcome.load_from_file()
succesfullwait = pd.DataFrame(index = data['software_events'].streams.WaitRewardOutcome.data.index, columns=['data'])
new_data = pd.json_normalize(data['software_events'].streams.WaitRewardOutcome.data['data'])
succesfullwait['data'] = new_data['IsSuccessfulWait'].values

label_dict = {
    "InterSite": '#808080',
    "Odor 1": '#7570b3',
    "Odor 2": '#1b9e77',
    "Odor 3": '#d95f02',
    "InterPatch": '#b3b3b3'}

def update_plot(x_start):
    fig, axs = plt.subplots(2,1, figsize=(20,8), gridspec_kw={'height_ratios': [2, 2]}, sharex=True)

    # sites_test = sites.loc[(sites.index > (zero_index + x_start))&(sites.index < (zero_index + x_start + 50))]   
    _legend = {}
    for idx, site in enumerate(sites.iloc[:-1].iterrows()):
        site_label = site[1]['data']["label"]
        if site_label == "Reward":
            site_label = f"Odor {site[1]['data']['odor']['index']+1}"
            facecolor = label_dict[site_label]
        elif site_label == "RewardSite":
            site_label = f"Odor {site[1]['data']['odor_specification']['index']+1}"
            facecolor = label_dict[site_label]
        elif site_label == "InterPatch":
            facecolor = label_dict[site_label]
        else:
            site_label = "InterSite"
            facecolor = label_dict["InterSite"]

        p = Rectangle(
            (sites.index[idx] - zero_index, -2), sites.index[idx+1] - sites.index[idx], 8,
            linewidth = 0, facecolor = facecolor, alpha = .5)
        
        _legend[site_label] = p
        axs[0].add_patch(p)
        
        q = Rectangle(
            (sites.index[idx] - zero_index, -2), sites.index[idx+1] - sites.index[idx], 8,
            linewidth = 0, facecolor = facecolor, alpha = .5)
        _legend[site_label] = q
        axs[1].add_patch(q)
        # axs[1].add_patch(p)
        
    s, lw = 400, 2
    # Plotting raster
    y_idx = -0.4
    _legend["Choice Tone"] = axs[1].scatter(choice_feedback.index - zero_index+0.2,
            choice_feedback.index * 0 + y_idx,
            marker="s", s=100, lw=lw, c='darkblue',
            label="Choice Tone")
    y_idx += 1
    _legend["Lick"] = axs[1].scatter(lick_onset.index - zero_index,
            lick_onset.index * 0 + y_idx,
            marker="|", s=s, lw=lw, c='k',
            label="Lick")
    _legend["Reward"] = axs[1].scatter(give_reward.index - zero_index,
            give_reward.index*0 + y_idx,
            marker=".", s=s, lw=lw, c='deepskyblue',
            label="Reward")
    _legend["Waits"] = axs[1].scatter(succesfullwait.index - zero_index,
        succesfullwait.index*0 + 1.2,
        marker=".", s=s, lw=lw, c='green',
        label="Reward")
    
    # _legend["Odor_on"] = axs.scatter(odor_on - zero_index,
    #     odor_on*0 + 2.5,
    #     marker="|", s=s, lw=lw, c='pink',
    #     label="ON")
    
    # _legend["Odor_off"] = axs.scatter(odor_off - zero_index,
    #     odor_off*0 + 2.5,
    #     marker="|", s=s, lw=lw, c='purple',
    #     label="ON")
    
    y_idx += 1

    #ax.set_xticks(np.arange(0, sites.index[-1] - zero_index, 10))
    axs[1].set_yticklabels([])
    axs[1].set_xlabel("Time(s)")
    axs[1].set_ylim(bottom=-1, top = 3)
    axs[1].grid(False)
    plt.gca().yaxis.set_visible(False)
    
    ax2 = axs[1].twinx()
    _legend["Velocity"] = ax2.plot(encoder_data.index - zero_index, encoder_data.filtered_velocity, c="k", label="Encoder", alpha = 0.8)[0]
    try:
        v_thr = config.streams.TaskLogic.data["operationControl"]["positionControl"]["stopResponseConfig"]["velocityThreshold"]
    except:
        v_thr = 8
    _legend["Stop Threshold"] = ax2.plot(ax2.get_xlim(), (v_thr, v_thr), c="k", label="Encoder", alpha = 0.5, lw = 2, ls = "--")[0]
    ax2.grid(False)
    ax2.set_ylim((-5, 70))
    ax2.set_ylabel("Velocity (cm/s)")
    
    ax3 = axs[0].twinx()
    if filtered:
        _legend["Breathing"] = ax3.plot(filtered_breathing.index - zero_index, filtered_breathing['data'].values, c="black", label="Breathing", alpha = 0.8)[0]
    else:
        _legend["Breathing"] = ax3.plot(breathing.index - zero_index, breathing.values, c="black", label="Breathing", alpha = 0.8)[0]

    ax3.grid(False)
    ax3.set_ylabel("Breathing")
    ax3.set_ylim((2100, 2400))
    axs[0].legend(_legend.values(), _legend.keys(), bbox_to_anchor=(1.1, 0.1), loc='center left', borderaxespad=0.)

    # axs[0].stairs(software_events.streams.RewardAvailableInPatch.data["data"].values[:-1],
    #           software_events.streams.RewardAvailableInPatch.data["data"].index.values -  zero_index,
    #           lw = 3, color = 'k', fill=0)
    
    for i in [0,1]:
        axs[i].set_xlabel("Time(s)")
        axs[i].grid(False)
        axs[i].set_ylim(bottom=-1, top = 4)
        axs[i].set_yticks([0,3])
        axs[i].yaxis.tick_right()
        axs[i].set_xlim([x_start, x_start + 10])
        
    plt.savefig(foraging_figures + f"\{x_start_widget.value}_time_detrended.svg", bbox_inches='tight', pad_inches=0.1, transparent=True)
    
# Define callback functions for the arrow buttons
def on_left_button_clicked(button):
    x_start_widget.value -= 5

def on_right_button_clicked(button):
    x_start_widget.value += 5

# Create arrow buttons
left_button = widgets.Button(description='◄')
right_button = widgets.Button(description='►')

# Define widget for the starting value of x-axis
x_start_widget = widgets.FloatText(value=00.0, description='X start:', continuous_update=False)

# Set button click event handlers
left_button.on_click(on_left_button_clicked)
right_button.on_click(on_right_button_clicked)

# Arrange the buttons and widget horizontally
button_box = widgets.HBox([left_button, right_button])
ui = widgets.VBox([button_box, x_start_widget])

# Create interactive plot
interactive_plot = widgets.interactive_output(update_plot, {'x_start': x_start_widget})

# Display the interactive plot and UI
display(ui, interactive_plot)

In [ ]:
df = lick_onset.reset_index()
# df.set_index('Time')
df['time_diff'] = df.Time.diff()
df['time'] = df.Time
df.set_index('Time', inplace=True)

In [ ]:
fig = plt.figure(figsize=(5, 4))
sns.histplot(df.time_diff, bins = 50, binrange=(0,0.6), color='darkgreen', element='step')
sns.despine()
plt.xlabel('Lick interval (s)')
plt.title(f'{mouse} - {session} - {reward_sites.loc[reward_sites.has_choice==True].shape[0]} trials')
print(f'Total flickers: {df.loc[df.time_diff <= 0.025].count()[0]} out of {df.count()[0]} licks ({df.loc[df.time_diff <= 0.025].count()[0]/df.count()[0]*100:.2f}%)')

In [ ]:
window = (-2,2)
trial_summary = plotting.trial_collection(reward_sites, df, mouse, session, taken_col='time_diff')

In [ ]:
sns.histplot(trial_summary.loc[~(trial_summary.water_onset >0)].speed, bins = 50, binrange=(0,0.5), color='red', element='step', alpha=0.5)
sns.histplot(trial_summary.loc[trial_summary.water_onset >0].speed, bins = 50, binrange=(0,0.5), color='darkgreen', element='step', alpha=0.5)

sns.despine()
plt.xlabel('Time difference between licks (s)')
plt.title(f'{mouse} - {session}')